In [ ]:
from mpi4py import MPI
import numpy
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from tqdm import tqdm
#NOTE: MAX IS 6,000,000 by 6,000,000 for c1exp, 1,000,000 by 1,000,000 for dsrl
fig = plt.figure()

comm = MPI.COMM_WORLD
size = comm.Get_size()
rank = comm.Get_rank()
stat = MPI.Status()

prob = 0.9
COLS = 400
ROWS = 198
generations = 600

if size > ROWS:
        print("Not enough ROWS")
        exit()
subROWS=ROWS//size+2

N=numpy.random.binomial(1,prob,size=(subROWS+2)*COLS)
M=numpy.reshape(N,(subROWS+2,COLS))

def msgUp(M):
        # Sends and Recvs rows with Rank+1
        dest = (rank+1)%size
        source = dest
#       print("MESSAGE UP: %d %d %d\n"%(rank,dest,source))
        comm.send(M[subROWS-2,:],dest=(rank+1)%size)
        M[subROWS-1,:]=comm.recv(source=(MPI.ANY_SOURCE))
        return 0

def msgDn(M):
        # Sends and Recvs rows with Rank-1
        dest  = (rank-1)%size
        source = dest
#       print("MeSSAGE DN: %d %d %d\n" % (rank,dest,source))
        comm.send(M[1,:],dest=(rank-1)%size)
        M[0,:] = comm.recv(source=(MPI.ANY_SOURCE))
#       M[0,:] = comm.recv(source=(rank-1)%size)
        return 0

def computeGridPoints(M):
        for ROWelem in range(1,subROWS-1):
                for COLelem in range(1,COLS-1):
                        sum = (M[ROWelem-1,COLelem-1]+M[ROWelem-1,COLelem]+M[ROWelem-1,COLelem+1]
                                +M[ROWelem,COLelem-1]+M[ROWelem,COLelem+1]
                                +M[ROWelem+1,COLelem-1]+M[ROWelem+1,COLelem]+M[ROWelem+1,COLelem+1])
                        if M[ROWelem,COLelem] == 1:
                                if sum < 2:
                                        intermediateM[ROWelem,COLelem] = 0
                                elif sum > 3:
                                        intermediateM[ROWelem,COLelem] = 0
                                else:
                                        intermediateM[ROWelem,COLelem] = 1
                        if M[ROWelem,COLelem] == 0:
                                if sum == 3:
                                        intermediateM[ROWelem,COLelem] = 1
                                else:
                                        intermediateM[ROWelem,COLelem] = 0
        return 0

generation = 0
ims=[]
for i in range(generations):
        generation = generation + 1
        intermediateM = numpy.copy(M)
        computeGridPoints(M)
        M = numpy.copy(intermediateM)
        #exhange edge rows for next interation
            #if rank == 0:
        #       msgUp(M)
        #elif rank == size-1:
        #       msgDn(M)
        #else:
        msgUp(M)
        msgDn(M)

        gatheredM=comm.gather(M[1:subROWS-1,:],root=0)
        if rank == 0:
                newGrid= numpy.vstack(gatheredM)
                result = numpy.vstack(newGrid)
                im=plt.imshow(result, animated=True,interpolation='None')
                ims.append([im])
        print(generation)
ani = animation.ArtistAnimation(fig, ims, interval=25, blit=True,repeat_delay=500)
ani.save('animate_life_0.9.mp4')
